In [6]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common import TimeoutException

from concurrent.futures import ThreadPoolExecutor
import threading
from threading import Thread
from functools import partial

import json
import time

In [2]:
def metaDta(url):
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    
    driver = webdriver.Firefox(options=options)
    driver.install_addon('ublock_origin-1.61.2.xpi', temporary=True)

    video = {}
    try:
        driver.get(url)
        print(url)

        # Attente jusqu'à ce que les métadonnées soient visibles
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'h1.ytd-watch-metadata'))
        )
        
        # Récupérer la durée de la vidéo
        try:
            duration = driver.find_element(By.XPATH, "//span[@class='ytp-time-duration']")
            duree = duration.text.split(":")

            if len(duree) == 2:  # Format 'm:ss'
                minutes, secondes = duree
                heures = "00"
            elif len(duree) == 3:  # Format 'hh:mm:ss'
                heures, minutes, secondes = duree
            dureeFinale = f"{int(heures):02}:{int(minutes):02}:{int(secondes):02}"
            video['duree'] = dureeFinale

        except Exception:
            print("Erreur de durée sur " + url)
            video['duree'] = None

        # Récupérer le genre
        try:
            genre = driver.find_element(By.XPATH, "//meta[@itemprop='genre']")
            video['genre'] = genre.get_attribute("content")
        except Exception:
            print("Erreur de genre sur " + url)
            video['genre'] = None

    except TimeoutException:
        print(f"Erreur : Timeout sur {url}")
        return None
    finally:
        driver.quit()
    
    return video

In [3]:
# Fonction de sauvegarde des données dans le fichier JSON
def sauvegarde_periodique(contenu, lock, stop_event, intervalle=90):
    while not stop_event.is_set():
        time.sleep(intervalle)
        with lock:
            print("Sauvegarde en cours...")
            with open("test.json", 'w', encoding='utf-8') as fichier:
                json.dump(contenu, fichier, indent=4, ensure_ascii=False)
            print("Sauvegarde effectuée.")

In [4]:
# Fonction pour traiter une URL et mettre à jour les résultats
def traiter_video(url, lock, contenu, counter):
    try:
        data = metaDta(url)
        if data:
            with lock:
                contenu[url].update(data)
            with lock:
                counter['i'] += 1
    except Exception as e:
        print(f"Erreur sur {url}: {e}")


In [ ]:
# Chargement des données existantes
with open("test.json", 'r', encoding='utf-8') as fichier:
    contenu = json.load(fichier)


videos = list(contenu.keys()) 
lock = threading.Lock() 
# Compteur pour suivre le progrès
counter = {'i': 0} 

# Drapeau pour signaler l'arrêt
stop_event = threading.Event()

# Lancement du thread de sauvegarde périodique
thread_sauvegarde = Thread(target=sauvegarde_periodique, args=(contenu, lock, stop_event), daemon=True)
thread_sauvegarde.start()

try :
    traiter_video_partial = partial(traiter_video, contenu=contenu, lock=lock, counter=counter)
    # Exécution concurrente pour accélérer le traitement
    with ThreadPoolExecutor(max_workers=5) as executor:
        # Commencer à la i-ème vidéo
        executor.map(traiter_video_partial, videos[counter['i']:])
finally:
    stop_event.set()
    thread_sauvegarde.join()
    print("Traitement terminé et sauvegarde arrêtée.")